In [1]:
import os
import torch
import copy

from PIL import Image
from llava.mm_utils import get_model_name_from_path
from llava.model.builder import load_pretrained_model
from llava.mm_utils import process_images, tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.conversation import conv_templates

In [2]:
def load_image(img_path):
    
    img = Image.open(img_path)
    return img

def make_heatmap(layer_attn, height_view, width_view):
    
    matrix = layer_attn.view(height_view, width_view)
    matrix = matrix.cpu().numpy()
    return matrix

In [ ]:
# input question and image path

qs = "what does it say in the bottom right corner?"
image_path = "./data/post-letter.jpg"
image = load_image(image_path)

model_path = os.path.expanduser("lmms-lab/llama3-llava-next-8b")
model_name = get_model_name_from_path(model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model("lmms-lab/llama3-llava-next-8b", None, model_name, attn_implementation="eager") # loading model and tokenizer

device = "cuda"
image_tensor = process_images([image], image_processor, model.config)
image_tensor = [_image.to(dtype=torch.float16, device=device) for _image in image_tensor]

conv_template = "llava_llama_3"
user_prompt = qs

question = DEFAULT_IMAGE_TOKEN + "\n {}".format(user_prompt) + "\n Answer briefly in only one sentence"

conv = copy.deepcopy(conv_templates[conv_template])
conv.append_message(conv.roles[0], question)
conv.append_message(conv.roles[1], None)
prompt_question = conv.get_prompt()

input_ids = tokenizer_image_token(prompt_question, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(device)
image_sizes = [image.size]

In [ ]:
with torch.inference_mode():
    # generating output_ids
    config, output_ids = model.generate(
        input_ids,
        images=image_tensor,
        image_sizes=image_sizes,
        do_sample=True,
        temperature=0.2,
        max_new_tokens=1024,
        use_cache=True,
        return_dict_in_generate=True,
        output_attentions=True,
        include_base_only=False,
        filter_newline=True)
    
outputs = tokenizer.batch_decode(output_ids['sequences'], skip_special_tokens=True)[0]

sub_crop_height = config[-2]
sub_crop_width = config[-1]

cum_token_averaged_attn = torch.zeros([1, sub_crop_height * sub_crop_width], device = "cpu")
img_token_idx = (input_ids == -200).nonzero(as_tuple=True)[1].item()

for output_attn in output_ids['attentions'][1:]:
    stacked_tensors = torch.stack(output_attn).cpu()
    stacked_tensors = stacked_tensors[24:32]
    mean_values = stacked_tensors.mean(dim=0)
    mean_values = mean_values.squeeze()
    average_values = mean_values.mean(dim=0)
    cum_token_averaged_attn += average_values[img_token_idx + 576: img_token_idx + 576 + sub_crop_height * sub_crop_width]

cum_token_averaged_attn = cum_token_averaged_attn/(len(output_ids['attentions'][1:]))
attn_matrix = make_heatmap(cum_token_averaged_attn, sub_crop_height, sub_crop_width) # output attention matrix

In [5]:
import pickle

list = []
list.append((attn_matrix))

with open('./data/output/attn.pkl', 'wb') as f:
    pickle.dump(list, f) # dumping the attention matrix into a pickle file